In [4]:
import pandas as pd
import os
import numpy as np
import datetime as dt
import re

In [5]:
# load city airbnb files including listings, reviews, availability calendars
pwd = os.getcwd()
city = 'boston'
listings_raw = pd.read_csv(pwd + '/' + city +'/listings.csv')
reviews_raw = pd.read_csv(pwd + '/' + city +'/reviews.csv')
calendar2017_raw = pd.read_csv(pwd + '/' + city +'/calendar_2016_2017.csv')
calendar2016_raw = pd.read_csv(pwd + '/' + city +'/calendar_2015_2016.csv')

print(listings_raw.shape)
print(reviews_raw.shape)
print(calendar2017_raw.shape)
print(calendar2016_raw.shape)
print(calendar2017_raw.head())
print(calendar2016_raw.head())

(2558, 92)
(68275, 6)
(1308890, 4)
(933670, 4)
   listing_id        date available price
0    12147973  2017-09-05         f   NaN
1    12147973  2017-09-04         f   NaN
2    12147973  2017-09-03         f   NaN
3    12147973  2017-09-02         f   NaN
4    12147973  2017-09-01         f   NaN
   listing_id        date available price
0     1810172  2015-10-03         f   NaN
1     1810172  2015-10-04         f   NaN
2     1810172  2015-10-05         f   NaN
3     1810172  2015-10-06         f   NaN
4     1810172  2015-10-07         f   NaN


In [3]:
calendar2017_raw = calendar2017_raw.drop_duplicates()
print(calendar2017_raw.date.describe())
print(calendar2017_raw.date.min())

count        1308525
unique           365
top       2017-07-20
freq            3585
Name: date, dtype: object
2016-09-06


In [4]:
calendar2016_raw = calendar2016_raw.drop_duplicates()
print(calendar2016_raw.date.max())

2016-10-01


In [5]:
# merge calendar files and remove duplicates
calendar2016_raw=calendar2016_raw[calendar2016_raw.date < calendar2017_raw.date.min()]
calendarall = pd.concat([calendar2017_raw, calendar2016_raw])
calendarall = calendarall.drop_duplicates()
print(calendarall.shape)

(2176812, 4)


In [6]:
# replace f with false and t with true
bool_map = {'f': False, 't': True}
calendarall['available'] = calendarall['available'].map(bool_map)
calendarall['date'] = pd.to_datetime(calendarall['date'], format="%Y-%m-%d")
calendarall['price'] = calendarall['price'].apply(lambda x: str(x).strip('$') if not pd.isnull(x) else x)
calendarall['price'] = calendarall['price'].apply(lambda x: str(x).replace(',','') if not pd.isnull(x) else x)
calendarall['price'] = pd.to_numeric(calendarall['price'])
calendarall.tail(15)
print(calendarall['price'].describe())
calendarall_available= calendarall[calendarall["available"] == True ]
calendarall_available.tail()

count    1.169709e+06
mean     1.939715e+02
std      1.642332e+02
min      1.000000e+01
25%      8.900000e+01
50%      1.500000e+02
75%      2.500000e+02
max      7.163000e+03
Name: price, dtype: float64


,listing_id,date,available,price
933640,6186360,2016-09-01,True,250.0
933641,6186360,2016-09-02,True,275.0
933642,6186360,2016-09-03,True,275.0
933643,6186360,2016-09-04,True,250.0
933644,6186360,2016-09-05,True,250.0


In [7]:
# check missing value
print(calendarall_available.isnull().sum())
calendar2016_peak=calendarall_available[calendarall_available.date <= '2016-9-30']
calendar2016_peak=calendar2016_peak[calendar2016_peak.date >='2016-7-1']
calendar2016_peak.head()
calendar2016_peak["date"].describe()

listing_id    0
date          0
available     0
price         0
dtype: int64


count                  146975
unique                     92
top       2016-09-05 00:00:00
freq                     1918
first     2016-07-01 00:00:00
last      2016-09-30 00:00:00
Name: date, dtype: object

In [8]:
# split to 2016 and 2017
calendarall_available_2016=calendarall_available[calendarall_available.date <='2016-9-30']
calendarall_available_2017=calendarall_available[calendarall_available.date >='2016-10-01']
#calendarall_available_2017=calendarall_available_2017[calendarall_available_2017.date <='2017-8-31']
calendarall_available_2016["date"].describe()

count                  551138
unique                    365
top       2016-09-05 00:00:00
freq                     1918
first     2015-10-02 00:00:00
last      2016-09-30 00:00:00
Name: date, dtype: object

In [9]:
# group calendar by listings for all availabile records 2016
calendar_ava_count_2016 = calendarall_available_2016.groupby(['listing_id'])['available'].count().reset_index()
calendar_ava_avg_2016 = calendarall_available_2016.groupby(['listing_id'])['price'].mean().reset_index()
print(calendar_ava_count_2016.shape)
print(calendar_ava_avg_2016.shape)
calendar_ava_avg_2016 = calendar_ava_avg_2016.rename(index=str, columns={"price": "price_avg_2016", "listing_id" : "id"})
calendar_ava_count_2016 = calendar_ava_count_2016.rename(index=str, columns={"available": "available_count_2016", "listing_id" : "id"})
calendarall_available_2016 = pd.merge(calendar_ava_avg_2016, calendar_ava_count_2016, on='id', how='outer')
calendarall_available_2016.head()

(3637, 2)
(3637, 2)


,id,price_avg_2016,available_count_2016
0,3353,50.000000,296
1,3781,150.000000,324
2,5453,150.000000,30
3,5506,147.307692,338
4,6695,198.385580,319


In [10]:
calendarall_available_2016["available_count_2016"].describe()

count    3637.000000
mean      151.536431
std       137.442810
min         1.000000
25%        15.000000
50%       107.000000
75%       304.000000
max       365.000000
Name: available_count_2016, dtype: float64

In [11]:
# group calendar by listings for all availabile records 2017
calendar_ava_count_2017 = calendarall_available_2017.groupby(['listing_id'])['available'].count().reset_index()
calendar_ava_avg_2017 = calendarall_available_2017.groupby(['listing_id'])['price'].mean().reset_index()
print(calendar_ava_count_2017.shape)
print(calendar_ava_avg_2017.shape)
calendar_ava_avg_2017 = calendar_ava_avg_2017.rename(index=str, columns={"price": "price_avg_2017", "listing_id" : "id"})
calendar_ava_count_2017 = calendar_ava_count_2017.rename(index=str, columns={"available": "available_count_2017", "listing_id" : "id"})
calendarall_available_2017 = pd.merge(calendar_ava_avg_2017, calendar_ava_count_2017, on='id', how='outer')
print(calendarall_available_2017.shape)

(2857, 2)
(2857, 2)
(2857, 3)


In [12]:
calendarall_available = pd.merge(calendarall_available_2016, calendar_ava_count_2017, left_on="id", right_on="id", how='left')

In [13]:
# merge 2016 and 2017 calendar 
print(calendarall_available.shape)

(3637, 4)


In [14]:
calendarall_available.isnull().sum()

id                         0
price_avg_2016             0
available_count_2016       0
available_count_2017    1234
dtype: int64

In [15]:
# group by listing id and count
listings_available_counts_2016_peak=calendar2016_peak.groupby('listing_id')['available'].count().reset_index()
listings_available_counts_2016_peak = listings_available_counts_2016_peak.rename(index=str, columns={"listing_id" : "id","available": "peak_available_count"})
calendarall_available = pd.merge(calendarall_available, listings_available_counts_2016_peak, on='id', how='left')
print(calendarall_available.shape)
calendarall_available.head()

(3637, 5)


,id,price_avg_2016,available_count_2016,available_count_2017,peak_available_count
0,3353,50.000000,296,249.0,67.0
1,3781,150.000000,324,NaN,67.0
2,5453,150.000000,30,NaN,30.0
3,5506,147.307692,338,333.0,76.0
4,6695,198.385580,319,322.0,69.0


In [16]:
reviews_raw['date'].max()

'2016-09-06'

In [17]:
reviews_2015_2016_all = reviews_raw[['listing_id','date']]
reviews_2015_2016_all = reviews_2015_2016_all[reviews_2015_2016_all.date <='2016-9-30'][reviews_2015_2016_all.date >='2015-10-1']
reviews_2015_2016_all['date'] = pd.to_datetime(reviews_2015_2016_all['date'], format="%Y-%m-%d")
reviews_2015_2016_all_count = reviews_2015_2016_all.groupby(['listing_id'])['date'].count().reset_index()
reviews_2015_2016_all_count = reviews_2015_2016_all_count.rename(index=str, columns={"date": "review_count_2016", "listing_id" : "id"})
calendarall_available = pd.merge(calendarall_available, reviews_2015_2016_all_count, on='id', how='left')
print(calendarall_available.shape)
calendarall_available.head()

(3637, 6)


,id,price_avg_2016,available_count_2016,available_count_2017,peak_available_count,review_count_2016
0,3353,50.000000,296,249.0,67.0,8.0
1,3781,150.000000,324,NaN,67.0,NaN
2,5453,150.000000,30,NaN,30.0,NaN
3,5506,147.307692,338,333.0,76.0,4.0
4,6695,198.385580,319,322.0,69.0,5.0


In [18]:
calendarall_available = calendarall_available.fillna(0)
calendarall_available.isnull().sum()
calendarall_available[calendarall_available['available_count_2016'] == 0]
calendarall_available[calendarall_available['available_count_2017'] == 0]

,id,price_avg_2016,available_count_2016,available_count_2017,peak_available_count,review_count_2016
1,3781,150.000000,324,0.0,67.0,0.0
2,5453,150.000000,30,0.0,30.0,0.0
6,8789,155.000000,281,0.0,67.0,0.0
29,19999,70.000000,320,0.0,67.0,0.0
36,23370,289.000000,339,0.0,67.0,0.0
39,24063,120.000000,304,0.0,67.0,0.0
44,27546,60.000000,295,0.0,67.0,0.0
50,34601,200.000000,309,0.0,67.0,0.0
51,37864,273.141447,304,0.0,61.0,0.0
72,65329,183.637821,312,0.0,67.0,0.0


In [19]:
print(listings_raw.shape)
print(listings_raw.columns)
print(listings_raw.describe())
print(listings_raw.head())

(2558, 92)
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'am

In [20]:
class CONST:
    SKIP = True
    NOSKIP = False
    
class Column:
    def __init__(self, name, coltype, isskip = CONST.NOSKIP, navalue = None, trans = None):
        self.name = name
        self.coltype = coltype
        self.navalue = navalue
        self.trans = trans
        self.isskip = isskip
    def getname(self):
        return self.name
    def gettype(self):
        return self.coltype
    def getnavalue(self):
        return self.navalue
    def gettrans(self):
        return self.trans
    def isskip (self):
        return self.isskip

In [21]:
col_0 = Column('id','number')
col_1 = Column('listing_url','url', CONST.SKIP)
col_2 = Column('scrape_id','string',CONST.SKIP)
col_3 = Column('last_scraped','date',CONST.SKIP)
col_4 = Column('name','string',CONST.SKIP)
col_5 = Column('summary', 'long_text',CONST.SKIP)
col_6 = Column('space', 'long_text',CONST.SKIP)
col_7 = Column('description', 'long_text',CONST.SKIP)
col_8 = Column('experiences_offered', 'long_text', CONST.SKIP)
col_9 = Column('neighborhood_overview','long_text', CONST.SKIP)
col_10 = Column('notes','long_text', CONST.SKIP)
col_11 = Column('transit','long_text', CONST.SKIP)
col_12 = Column('thumbnail_url','url', CONST.SKIP)
col_13 = Column('medium_url','url', CONST.SKIP)
col_14 = Column('picture_url','url', CONST.SKIP)
col_15 = Column('xl_picture_url','url', CONST.SKIP)
col_16 = Column('host_id','string', CONST.SKIP)
col_17 = Column('host_url','url', CONST.SKIP)
col_18 = Column('host_name','string', CONST.SKIP)
col_19 = Column('host_since','date', CONST.SKIP,'today','days')
col_20 = Column('host_location','location',CONST.SKIP)
col_21 = Column('host_about','string',CONST.SKIP)
col_22 = Column('host_response_time','category', CONST.NOSKIP,'mode')
col_23 = Column('host_response_rate','percentage',CONST.NOSKIP,'mode')
col_24 = Column('host_acceptance_rate','percentage',CONST.NOSKIP,'mode')
col_25 = Column('host_is_superhost','boolean',CONST.NOSKIP,'false')
col_26 = Column('host_thumbnail_url','url',CONST.SKIP)
col_27 = Column('host_picture_url','url',CONST.SKIP)
col_28= Column('host_neighbourhood','category', CONST.SKIP)
col_29 = Column('host_listings_count','number',CONST.SKIP, 'zero')
#host_total_listings_count is duplicated with host_listings_count
col_30 = Column('host_total_listings_count','number',CONST.SKIP)
col_31 = Column('host_verifications','array_category', CONST.SKIP)
col_32 = Column('host_has_profile_pic','boolean', CONST.NOSKIP,'false')
col_33 = Column('host_identity_verified','boolean', CONST.NOSKIP,'false')
col_34 = Column('street','location', CONST.SKIP)
col_35 = Column('neighbourhood','category', CONST.SKIP)
col_36 = Column('neighbourhood_cleansed','category')
col_37 = Column('neighbourhood_group_cleansed','category', CONST.SKIP)
col_38 = Column('city','location', CONST.SKIP)
col_39 = Column('state','location', CONST.SKIP)
col_40 = Column('zipcode','location', CONST.SKIP)
col_41 = Column('market','location', CONST.SKIP)
col_42 = Column('smart_location','location', CONST.SKIP)
col_43 = Column('country_code','location', CONST.SKIP)
col_44 = Column('country','location', CONST.SKIP)
col_45 = Column('latitude','location', CONST.NOSKIP)
col_46 = Column('longitude','location', CONST.NOSKIP)
col_47 = Column('is_location_exact','boolean', CONST.SKIP)
col_48 = Column('property_type','category', CONST.NOSKIP, 'Other')
col_49 = Column('room_type','category')
col_50 = Column('accommodates','number', CONST.NOSKIP,'one')
col_51 = Column('bathrooms','number', CONST.NOSKIP, 'zero')
col_52 = Column('bedrooms','number', CONST.NOSKIP,'zero')
col_53 = Column('beds','number', CONST.NOSKIP,'zero')
col_54 = Column('bed_type','category')
col_55 = Column('amenities','array_category')
col_56 = Column('square_feet','number', CONST.SKIP)
col_57 = Column('price','dollar')
col_58 = Column('weekly_price','dollar', CONST.SKIP) # TODO
col_59 = Column('monthly_price','dollar', CONST.SKIP) # TODO
col_60 = Column('security_deposit','dollar', CONST.NOSKIP, 'zero')
col_61 = Column('cleaning_fee','dollar', CONST.NOSKIP, 'zero')
col_62 = Column('guests_included','number', CONST.NOSKIP, 'zero')
col_63 = Column('extra_people','dollar', CONST.NOSKIP, 'zero')
col_64 = Column('minimum_nights','number', CONST.NOSKIP, 'min')
col_65 = Column('maximum_nights','number', CONST.NOSKIP, 'max')
col_66 = Column('calendar_updated','category')
col_67 = Column('has_availability','boolean', CONST.SKIP)
col_68 = Column('availability_30','number')
col_69 = Column('availability_60','number')
col_70 = Column('availability_90','number')
col_71 = Column('availability_365','number')
col_72 = Column('calendar_last_scraped','date', CONST.SKIP)
col_73 = Column('number_of_reviews','number', CONST.NOSKIP, 'zero')
col_74 = Column('first_review','date', CONST.SKIP)
col_75 = Column('last_review','date', CONST.SKIP)
col_76 = Column('review_scores_rating','number', CONST.NOSKIP,'mean')
col_77 = Column('review_scores_accuracy','number', CONST.NOSKIP,'mean')
col_78 = Column('review_scores_cleanliness','number', CONST.NOSKIP,'mean')
col_79 = Column('review_scores_checkin','number', CONST.NOSKIP,'mean')
col_80 = Column('review_scores_communication','number', CONST.NOSKIP,'mean')
col_81 = Column('review_scores_location','number', CONST.NOSKIP,'mean')
col_82 = Column('review_scores_value','number', CONST.NOSKIP,'mean')
col_83 = Column('requires_license','boolean',CONST.SKIP)
col_84 = Column('license','string', CONST.SKIP)
col_85 = Column('jurisdiction_names', 'string', CONST.SKIP)
col_86 = Column('instant_bookable','boolean')
col_87 = Column('cancellation_policy','category')
col_88 = Column('require_guest_profile_picture','boolean')
col_89 = Column('require_guest_phone_verification','boolean')
col_90 = Column('calculated_host_listings_count','number', CONST.SKIP)
col_91 = Column('reviews_per_month','number', CONST.NOSKIP,'zero')
#print(eval('col_' + str(columns_raw.get_loc('id'))).gettype())

In [22]:
print( listings_raw.columns.get_loc('reviews_per_month'))
print( listings_raw.columns.shape)

91
(92,)


In [23]:
def df_type_trans(df):
    global columns
    newcolumns = []
    columns = []
    columns_raw = df.columns
    for i in range(0,len(columns_raw)):
        column = eval('col_' + str(i))
        t = column.getname()
        if column.gettype() == 'string' or column.gettype() == 'long_text':
            df[t] = df[t].astype(str)
        elif column.gettype() == 'date':
            df[t] = pd.to_datetime(df[t], format="%Y-%m-%d")
            if column.getnavalue() == 'today':
                df[t].fillna(dt.datetime.today(), inplace = True)
        elif column.gettype() == 'percentage':
            number_re = re.compile(r'[%]')
            str2float = lambda x: x if np.isreal(x) else number_re.sub('', x, re.U)
            df[t] = df[t].map(str2float).astype(float)
            if column.getnavalue() == 'mode':
                df[t].fillna(df[t].mode()[0], inplace = True)
        elif column.gettype() == 'number':
            df[t] = pd.to_numeric(df[t])
            if  column.getnavalue() =='zero':
                df[t].fillna(0, inplace = True)
            elif column.getnavalue() == 'mean':
                df[t].fillna(df[t].mean(), inplace = True)
            elif column.getnavalue() =='min':
                df[t].fillna(df[t].min(), inplace = True)
            elif column.getnavalue() =='max':
                df[t].fillna(df[t].max(), inplace = True)
            elif column.getnavalue() == 'mode':
                df[t].fillna(dt[t].mode()[0], inplace = True)
        elif column.gettype() == 'dollar':
            number_re = re.compile(r'[$,]')
            str2float = lambda x: x if np.isreal(x) else number_re.sub('', x, re.U)
            df[t] = df[t].map(str2float).astype(float)
            if column.getnavalue() == 'mode':
                df[t].fillna(dt[t].mode()[0], inplace = True) 
            elif  column.getnavalue() =='zero':
                df[t].fillna(0, inplace = True)
        elif column.gettype() == 'boolean':
            bool_map = {'f': 0, 't': 1}
            df[t] = df[t].map(bool_map)  
            if column.getnavalue() == 'false':
                df[t].fillna(0, inplace = True)
            df[t] = pd.to_numeric(df[t], downcast='integer')
        elif column.gettype() == 'category':
            if column.getnavalue() == 'mode':
                df[t].fillna(df[t].mode()[0], inplace = True)
            elif column.getnavalue() is not None:
                df[t].fillna(column.getnavalue(), inplace = True)
        elif column.gettype() == 'array_category':
            if column.getnavalue() == 'mode':
                df[t].fillna(df[t].mode()[0], inplace = True)
            array_feature = []
            df[t] = df[t].map(
                lambda features: ",".join([re.sub("([^a-zA-Z0-9,])", "", "".join( re.sub("([{}\"\s+])", "", "".join(feature)))) for feature in features.split(",")])
            )
            array_feature = np.unique(np.concatenate(df[t].map(lambda features: features.split(","))))[1:]
            array_matrix = np.array([df[t].map(lambda values: 1 if feature in values else 0).values for feature in array_feature])
            df = pd.concat([df, pd.DataFrame(data=array_matrix.T, columns=array_feature)], axis=1)
#             for feature in array_feature:
#                 newcolumns.append(feature)  
#                 print(type(df[feature][0] == False))
#                 df[feature] = df[feature].apply(lambda x : 0 if x == False else 1)
#                 df[feature] = pd.to_numeric(df[feature])
        if column.gettrans() == 'days':
            t_new = t + '_days'
            #df[t_new] = dt.datetime.today()-df[t]
            df[t_new] = df[t].apply(lambda x: (dt.datetime.today()-x).days)
            #if not column.isskip:
            columns.append(t_new)   
            
        if not column.isskip and not column.gettype() == 'array_category':
            print('%i: %s, Type: %s, Fill Na: %s, NA: %i' % (i, t, column.gettype(),column.getnavalue(),df[t].isnull().sum()))
            columns.append(t) 
        #df[newcolumns] = df[newcolumns].apply(lambda x : 0 if x == False else 1)
    return df

In [24]:
listings = df_type_trans(listings_raw)

0: id, Type: number, Fill Na: None, NA: 0
22: host_response_time, Type: category, Fill Na: mode, NA: 0
23: host_response_rate, Type: percentage, Fill Na: mode, NA: 0
24: host_acceptance_rate, Type: percentage, Fill Na: mode, NA: 0
25: host_is_superhost, Type: boolean, Fill Na: false, NA: 0
32: host_has_profile_pic, Type: boolean, Fill Na: false, NA: 0
33: host_identity_verified, Type: boolean, Fill Na: false, NA: 0
36: neighbourhood_cleansed, Type: category, Fill Na: None, NA: 0
45: latitude, Type: location, Fill Na: None, NA: 0
46: longitude, Type: location, Fill Na: None, NA: 0
48: property_type, Type: category, Fill Na: Other, NA: 0
49: room_type, Type: category, Fill Na: None, NA: 0
50: accommodates, Type: number, Fill Na: one, NA: 0
51: bathrooms, Type: number, Fill Na: zero, NA: 0
52: bedrooms, Type: number, Fill Na: zero, NA: 0
53: beds, Type: number, Fill Na: zero, NA: 0
54: bed_type, Type: category, Fill Na: None, NA: 0
57: price, Type: dollar, Fill Na: None, NA: 0
60: securit

In [25]:
print(listings.shape)
columns = columns + ["AirConditioning","BuzzerWirelessIntercom","CableTV","CarbonMonoxideDetector","Dryer","ElevatorinBuilding","Essentials","FamilyKidFriendly","FireExtinguisher","FirstAidKit","FreeParkingonPremises","IndoorFireplace","Petsliveonthisproperty","SafetyCard","Shampoo","SmokeDetector","TV","Washer"]
print((columns))
listings.head()
#for column in columns:
#    print(column)
#print(listings.property_type.describe())
#print(listings.host_response_time.isnull().sum())
#print(listings.host_response_time.mode()[0])

(2558, 141)
['id', 'host_since_days', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification', 'reviews_per_month', 'AirConditioning', 'BuzzerWirelessIntercom', 'CableTV', 'CarbonMonoxideDetector', 'Dryer', 'ElevatorinBuilding', 'Essentials', 'Fa

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,SafetyCard,Shampoo,SmokeDetector,SmokingAllowed,SuitableforEvents,TV,Washer,WasherDryer,WheelchairAccessible,WirelessInternet
0,1810172,https://www.airbnb.com/rooms/1810172,20151002231814,2015-10-03,Room for 2 in Rozzie - Full Bath,This second floor bedroom sleeps two in a quee...,nan,This second floor bedroom sleeps two in a quee...,none,Our neighborhood is quiet and relaxed. There i...,...,0,1,1,0,0,1,1,0,0,1
1,6976,https://www.airbnb.com/rooms/6976,20151002231814,2015-10-03,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...",This is a well-maintained two-family house bui...,"Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,1,1,1,0,0,1,1,0,0,1
2,3075044,https://www.airbnb.com/rooms/3075044,20151002231814,2015-10-03,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,0,1,1,0,0,1,1,0,0,1
3,4283698,https://www.airbnb.com/rooms/4283698,20151002231814,2015-10-03,Private 1 bedroom w/ kitchenette,"This is a fantastic, newly updated space. Lar...",This is a fantastic space! We've finished all...,"This is a fantastic, newly updated space. Lar...",none,We're just a few blocks from the shops and res...,...,0,0,1,0,0,0,0,0,0,1
4,4085362,https://www.airbnb.com/rooms/4085362,20151002231814,2015-10-03,"Sub""Urban"" Vacationing Rm 2",Come share our beautifully maintained grand t...,We provide a bedroom and a full shared bath fo...,Come share our beautifully maintained grand t...,none,Roslindale is one of the neighborhoods of Bost...,...,1,0,1,0,0,1,1,0,0,1


In [26]:
print(calendarall_available.shape)
print(listings.shape)
listings_bak = pd.merge(listings[columns], calendarall_available, on="id", how='inner')
listings_bak.describe()
listings_bak.shape

(3637, 6)
(2558, 141)


(2437, 66)

In [27]:
listings_bak = listings_bak.rename(index=str, columns={"neighbourhood_cleansed": "neighbourhood"})

In [28]:
listings_bak.isnull().sum()

id                                  0
host_since_days                     0
host_response_time                  0
host_response_rate                  0
host_acceptance_rate                0
host_is_superhost                   0
host_has_profile_pic                0
host_identity_verified              0
neighbourhood                       0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                           0
bedrooms                            0
beds                                0
bed_type                            0
price                               0
security_deposit                    0
cleaning_fee                        0
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
calendar_updated                    0
availability

In [29]:
listings_bak.to_csv(pwd + '/' + city +'/listings_2015.csv', index = False, encoding='utf-8')

In [30]:
listings_bak.columns

Index(['id', 'host_since_days', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'calendar_updated', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'reviews_per_month', 'AirConditioning', 'BuzzerWirelessIntercom',
       'C